###  The Chaperone Effect in Scientific Publication

    When we do job hunting, we always ask should I go to a big name or startup company? We ask this question because we care about our future success. However, industry field success is hard to track because people rarely share their work experience or salaries. 

    Luckily tracking success in the academic field is relatively easy. A critical indicator of success is the number of scientific publications. People are encouraged to publish findings in scientific journals. Most of the time, the endpoint of a project is to publish papers. In scientific publications, generally, all the non-last authors are ordered by their contributions. And the last author is the corresponding author, the Principal Investigator (PI) of the paper. Whether a person published paper(s) as PI is a strong indicator of academic field success. 

    This project tends to answer the following question:
        1. Who is more comfortable to publish a paper as PI, a non-last author who has published paper(s) on that journal (chaperone effect), or those who never publish any articles on that journal?
        2. How many papers a non-last author need to publish to become chaperoned?
        3. Who will publish more papers in the future, new or chaperoned PI?
        4. A miscellaneous question: What's the countrywide performance of question 1 to 3.

    To answer these questions, all the research publications (67628 publications) from top-rated journals: Nature (26630 publications), Cell (14218 publications), and Science (26780 publications), were collected from EndnoteX9 software, specifically from the database PubMed. 
    
    PIs published papers between 2000 and 2020 (18926 from Science, 19491 from Nature, 9749 from Cell) will be labeled as 
        1. Established: published at least one PI papers earlier within the same journal
        2. New: never published articles within that journal
        3. Chaperoned: published papers as non-PI before posting a PI paper at that year
    
    This notebook mainly focused on data cleaning and author labeling. Results were visualized through Tableau Public(https://public.tableau.com/profile/jk4983#!/vizhome/ChaperoneinJournalPublications/Dashboard3).

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

### Data cleaning

    There are various types of publications in Science. Only research papers (with abstracts) will be collected. Other documents with empty abstract will be dropped through df.dropna() while reading data.

In [2]:
colnames = ['Author', 'Year', 'Date', 'Title', 'Journal', 'Abstract','URL','Notes', 'Address']
df = pd.read_csv('SCIENCE FULL.csv', header=None, names = colnames).dropna().reset_index(drop = True)
df.head(3)

,Author,Year,Date,Title,Journal,Abstract,URL,Notes,Address
0,"Z. GALCHEVA-GARGOVA, K. N. KONSTANTINOV, I. H....",1996,Jun 21,Binding of zinc finger protein ZPR1 to the epi...,Science,ZPR1 is a zinc finger protein that binds to th...,https://www.ncbi.nlm.nih.gov/pubmed/8650580,Galcheva-Gargova Z;Konstantinov K N;Wu I H...,Department of Biochemistry and Molecular Biolo...
1,"A. GENIN, J. S. JAFFE, R. REEF, C. RICHTER and...",2005,May 6,Swimming against the flow: a mechanism of zoop...,Science,Zooplankton reside in a constantly flowing env...,https://www.ncbi.nlm.nih.gov/pubmed/15879218,Genin Amatzia;Jaffe Jules S;Reef Ruth;Rich...,Interuniversity Institute for Marine Sciences ...
2,"K. W. NG, N. FAULKNER, G. H. CORNISH, A. ROSA,...",2020,Dec 11,Preexisting and de novo humoral immunity to SA...,Science,Zoonotic introduction of novel coronaviruses m...,https://www.ncbi.nlm.nih.gov/pubmed/33159009,Ng Kevin W;Faulkner Nikhil;Cornish Georgin...,"Retroviral Immunology, The Francis Crick Insti..."


### Take a look at authors address

In [3]:
print(df.Address[3])
print('')
print(df.Address[50])
print('')
print(df.Address[100])
print('')
print(df.Address[200])
print('')
print(df.Address[500])
print('')
print(df.Address[1000])
print('')
print(df.Address[2000])
print('')
print(df.Address[3000])

Department of Physics and Astronomy, Bucknell University, Lewisburg, PA 17837-2029, USA. katelynallers@gmail.com.;Department of Astrophysics, American Museum of Natural History, New York, NY 10024-5102, USA.;Scottish Universities Physics Alliance, Institute for Astronomy, University of Edinburgh, Edinburgh EH9 3HJ, UK.;Centre for Exoplanet Science, University of Edinburgh, Edinburgh EH9 3FD, UK.;Center for Astrophysics, Harvard Smithsonian, Cambridge, MA 02138-1516, USA.;American Astronomical Society, Washington, DC 20006-1681, USA.

Department of Molecular Biology, University of Texas Southwestern Medical Center, Dallas, TX 75390, USA.

European Commission, Directorate General Joint Research Centre, Institute for the Protection and Security of the Citizen, Agriculture and Fisheries Unit, TP 051, 21020, Italy. franz.hoelker@jrc.it

Department of Genetics, Hospital for Sick Children, Toronto, Ontario, Canada.

JILA, University of Colorado and National Institute of Standards and Technolo

### Note
     One publication may have one or more addresses. The last address is the PI's address, which will be kept for further analysis.
    To answer the question "What's the countrywide performance of question 1 to 3?", countries will be extracted from the PI's address.

### Country

    The following example shows how the PI's Country was extracted: 
  
    Given:  
     
    Address 1: 
    DEPARTMENT OF PHYSICS AND ASTRONOMY, BUCKNELL UNIVERSITY, LEWISBURG, PA 17837-2029, USA. KATELYNALLERS@GMAIL.COM.;DEPARTMENT OF ASTROPHYSICS, AMERICAN MUSEUM OF NATURAL HISTORY, NEW YORK, NY 10024-5102, USA.;SCOTTISH UNIVERSITIES PHYSICS ALLIANCE, INSTITUTE FOR ASTRONOMY, UNIVERSITY OF EDINBURGH, EDINBURGH EH9 3HJ, UK.;CENTRE FOR EXOPLANET SCIENCE, UNIVERSITY OF EDINBURGH, EDINBURGH EH9 3FD, UK.;CENTER FOR ASTROPHYSICS, HARVARD SMITHSONIAN, CAMBRIDGE, MA 02138-1516, USA.;AMERICAN ASTRONOMICAL SOCIETY, WASHINGTON, DC 20006-1681, USA.
    
    Address 2:
    IBM RESEARCH DIVISION, T. J. WATSON RESEARCH CENTER, YORKTOWN HEIGHTS, NY 10598, USA. JBHANNON@US.IBM.COM
    
    Step 1 will get PI's address:
    AMERICAN ASTRONOMICAL SOCIETY, WASHINGTON, DC 20006-1681, USA.
    IBM RESEARCH DIVISION, T. J. WATSON RESEARCH CENTER, YORKTOWN HEIGHTS, NY 10598, USA. JBHANNON@US.IBM.COM
    
    Step 2 will get the element that contains the PI's Country:
    USA.
    USA. JBHANNON@US.IBM.COM
    
    Step 3 only keep Country's name:
    USA
    USA
    
    Step 4: only keep Characters from A to Z if there are numbers or punctuations in the Country.  
    
    Step 5 is the most time-consuming as it is the 'cleaning' step.
    After steps 1-4, country names are printed out, and countries with wrong names will be corrected manually in this step.
    
    The detailed steps were showed below:  

In [4]:
df['Address'] = df['Address'].str.upper() 

#step1: only keep the last authors address (PI's address)
df['PI_Address'] = df['Address'].apply(lambda x: x.split(';')[-1]) 

#step2: get the element contains PI's country
df['Country'] = df['PI_Address'].apply(lambda x: x.split(', ')[-1])  

# step3: split by '.' to remove email info 
df['Country'] = df['Country'].apply(lambda x: x.split('.')[:1][0]) 

# step4: only keep charactors 
import re
df['Country'] = df['Country'].apply(lambda x: re.sub(r"[^A-Z]","",x))

#step5: Country info cleaning
from re import search
for i in range(df.shape[0]):
    if df['Country'][i].find('USA') != -1:
        df['Country'][i] = 'USA'

    if df['Country'][i].find('CANADA') != -1:
        df['Country'][i] = 'CANADA'
    if df['Country'][i] in ['TORONTO']:
        df['Country'][i] = 'CANADA'        
        
    if df['Country'][i].find('NETHERLAND') != -1:
        df['Country'][i] = 'NETHERLANDS'
    if df['Country'][i].find('JAPAN') != -1:
        df['Country'][i] = 'JAPAN'
    if df['Country'][i].find('KOREA') != -1:
        df['Country'][i] = 'KOREA'
    if df['Country'][i] == 'SEOULNATIONALUNIVERSITY':
        df['Country'][i] = 'KOREA'    
        
    if df['Country'][i].find('CHINA') != -1:
        df['Country'][i] = 'CHINA'
    if df['Country'][i].find('HONGKONG') != -1:
        df['Country'][i] = 'CHINA'
        
    if df['Country'][i] in ['HONGKONG', 'TAIWAN', 'P', 'PR','PRC', 'BEIJING','PEOPLES']:
        df['Country'][i] = 'CHINA'
    if df['Country'][i].find('RUSSIA') != -1:
        df['Country'][i] = 'RUSSIA'

    if df['Country'][i].find('SINGAPORE') != -1:
        df['Country'][i] = 'SINGAPORE' 
    if df['Country'][i].find('FRANCE') != -1:
        df['Country'][i] = 'FRANCE' 
    if df['Country'][i].find('PARIS') != -1:
        df['Country'][i] = 'FRANCE'
    
    if df['Country'][i].find('GERMANY') != -1:
        df['Country'][i] = 'GERMANY' 
    if df['Country'][i] in ['HEIDELBERG','BERLIN']:
        df['Country'][i] = 'GERMANY'        
        
    
    if df['Country'][i].find('GEORGIA') != -1:
        df['Country'][i] = 'GEORGIA'     
    
    
    if df['Country'][i].find('PANAMA') != -1:
        df['Country'][i] = 'PANAMA'   
    if df['Country'][i].find('SAUDIARABIA') != -1:
        df['Country'][i] = 'SAUDI ARABIA'        
        
    if df['Country'][i] == 'UNITEDARABEMIRATES':
        df['Country'][i] = 'UNITED ARAB EMIRATES'
    if df['Country'][i] == 'BRASIL':
        df['Country'][i] = 'BRAZIL'       
         
    if df['Country'][i].find('UK') != -1:
        df['Country'][i] = 'UK' 
    if df['Country'][i] in ['UNITEDKINGDOM','UNITEDKINGDON', 'CAMBRIDGE','ENGLAND', 'SCOTLAND', 'WALES', 'NORTHERNIRELAND']:
        df['Country'][i] = 'UK'
    if df['Country'][i] in ['KISUMU']:
        df['Country'][i] = 'KENYA'
    if df['Country'][i] in ['UNIVERSIDADNACIONALA']:
        df['Country'][i] = 'COLOMBIA'  
    if df['Country'][i] in ['BLOOMINGTON']:
        df['Country'][i] = 'INDANA' 
         
    if df['Country'][i] in ['AARHUSUNI']:
        df['Country'][i] = 'DENMARK'
        
    if df['Country'][i] == 'NEWZEALAND':
        df['Country'][i] = 'NEW ZEALAND'  
    
    if df['Country'][i] == 'CZECHREPUBLIC':
        df['Country'][i] = 'CZECH REPUBLIC'  
    if df['Country'][i].find('SOUTHAFRICA') != -1:
        df['Country'][i] = 'SOUTH AFRICA'  
        
    if df['Country'][i] in ['HOWARDHUGHESMEDICALINSTITUTE', 'UCLASCHOOLOFMEDICINE','CALIFORNIA', 'ILLINOIS',
                            'UNITEDSTATES', 'TEXAS','MARYLAND','INSTITUTEOFGEOPHYSICSANDPLANETARY','CALIFORNIAI',
                            'UNIVERSITYOFTEXASATAUSTIN','GALVESTON','BIRMINGHAM','SEATTLE','SANTABARBARA',
                            'UNIVERSITYOFCALIFORNIAATSANFRANCISCO','UNIVERSITYOFCALIFORNIAATSANFRANCISCOUCSF',
                            'UNIVERSITYOFCALIF','NEWYORK','BOSTON','IRVINE','HOUSTON','IOWACITY','ITHACANY',
                            'SANFRANCISCO','SANTACRUZ','WORCESTER','OMAHA','DALLAS','SANDIEGO','CALI',
                            'LAWRENCELIVERMORENATIONALL','RI','MADISON','BOULDER','BERKELE','UNIVERSITYOFWISCONSINMADISON',
                            'UNIVERSITYOFCALIFORNIAATBERKELEY','CHAPELHILL','RIVERSIDE','AUSTIN','LAJOLLA','MINNEAPOLIS',
                            'CHARLOTTESVILLE','CHILDRENSHOSPITALOFPHILADELPHIA','SALTLAKECITY','EASTLANSING','DENVER',
                            'RALEIGH','CALIFORNIAINSTITUTEOFTECH','PISCATAWAY','UCLA','BERKELY','BERKELEY','GAINESVILLE','RENO',
                            'CHARLESTOWN','LEXINGTON','SOUTHSANFRANCISCO','LOSANGELES','UNIVERSITYOFCALIFORNIA',
                            'ATLANTA','PASADENA','ANNARBOR','EUGENE','STONYBROOK',
                            'HI','PA','OH','CO','CA','MA','MD','NY','NH','MN','TN','MI','NH','MN','DC','IL',
                            'IN','WI','NJ','TX','NC','MO','VA','WA','CT','US','DE','UTAH','AL','FL','DAVIS',
                           'NM', 'LA','OR','GA','OK','NE','AZ']:
        df['Country'][i] = 'USA'            

#df.Country.unique()
print(df.Country.value_counts()[:50])

USA               16462
UK                 1963
GERMANY            1452
JAPAN               942
FRANCE              839
CANADA              639
SWITZERLAND         629
CHINA               506
NETHERLANDS         363
AUSTRALIA           330
ISRAEL              236
ITALY               221
SWEDEN              215
SPAIN               170
AUSTRIA             148
DENMARK             138
BELGIUM             101
KOREA                98
SINGAPORE            59
NORWAY               56
FINLAND              49
NEW ZEALAND          45
INDIA                44
RUSSIA               42
PHILADELPHIA         39
SOUTH AFRICA         36
BRAZIL               35
HUNGARY              28
IRELAND              27
PORTUGAL             20
GREECE               20
ARGENTINA            19
SAUDI ARABIA         17
CZECH REPUBLIC       16
PORTLAND             14
POLAND               14
U                    13
MEXICO               13
PANAMA               12
CHILE                12
ICELAND              11
UNIVERSITYPARK  

### note
    After these cleaning steps, the country info is still not very 'clean'. However, the 'dirty' data only have few observations. I will leave them here then clean them if necessary. 

## Author 

    Both attribute 'Notes' and 'Author' have all the author names. 

In [5]:
df['Year'] = df['Year'].apply(lambda x: int(x)) #change str to int
print('** Author format after 2002 ** ')
print(df.loc[df['Year'] > 2002].reset_index(drop = True).Author[4])
print('')
print('**Notes format after 2002** ')
print(df.loc[df['Year'] > 2002].reset_index(drop = True).Notes[4])
print('')
print('** Author format before 2002 ** ')
print(df.loc[df['Year'] < 2002].reset_index(drop = True).Author[1])
print('')
print('** Author format before 2002 **')
print(df.loc[df['Year'] < 2002].reset_index(drop = True).Notes[1])

** Author format after 2002 ** 
T. WEBER, S. JOHN, A. TAGLIABUE and T. DEVRIES

**Notes format after 2002** 
 Weber  Thomas;John  Seth;Tagliabue  Alessandro;DeVries  Tim;eng;Research Support  Non-U.S. Gov't;Research Support  U.S. Gov't  Non-P.H.S.;Science. 2018 Jul 6;361(6397):72-76. doi: 10.1126/science.aap8532.

** Author format before 2002 ** 
L. M. SANCHEZ, A. J. CHIRINO and PJ BJORKMAN

** Author format before 2002 **
 Sanchez  L M;Chirino  A J;Bjorkman  P j;eng;Science. 1999 Mar 19;283(5409):1914-9. doi: 10.1126/science.283.5409.1914.


### Note    
    'Author' attribute provided author's name by last name, first initial and/or middle initial, it may run risk to identify authors using this naming format as distinct individuals may share the same formatted name, ie, Zhang Zixuan and Zhang Zheng share same formated name 'Zhang Z.'. 

    Therefore, I tend to use 'Notes' to get author's full name, however, I observed a strange phenomenon: in previous notebook in 2002 there was a big surge of 'New' PIs as almost all the PIs were labeled as 'New' (fig below). There are definitely something wrong. 

    So what happened in 2002? 
    Besides printing out records to check, I also found this:

    It was mentioned in the publication "Torvik VI, Smalheiser NR (2009) Author name disambiguation in MEDLINE. ACM Trans Knowl Discov Data 3:1–29.": Author names have traditionally been encoded in MEDLINE by last name, first initial, middle initial (when available), and suffix (when available), whereas full first names have been included (when available) only since 2002. 

    This means the format of author names had a big change in year 2002, and full name were only provided in 'Notes' after year 2002. As the format of 'Note' is not consistent over time, I have to use 'Author' attribute to get individuals names although it is not the best resource.
    
    C'est la vie

<img src='image.png' width="800" height="400">

In [6]:
print(df.Author[1])
print('')
print(df.Author[4])

A. GENIN, J. S. JAFFE, R. REEF, C. RICHTER and P. J. FRANKS

L. M. SANCHEZ, A. J. CHIRINO and PJ BJORKMAN


    PI (the last author) and other authors are separated by 'and', they could be extracted using split function 

In [7]:
# PI (last author) and other authors are separated by and
# get other authors name
df['OtherAuthor'] = df['Author'].apply(lambda x: x.split("and")[0])
# get PI's name #use strip() to remove leading and ending blanks 
df['PI'] = df['Author'].apply(lambda x: x.split("and")[-1].strip())

print(df.OtherAuthor[1])
print('')
print(df.OtherAuthor[4])
print('')
print(df.PI[1])
print('')
print(df.PI[2])

A. GENIN, J. S. JAFFE, R. REEF, C. RICHTER 

L. M. SANCHEZ, A. J. CHIRINO 

P. J. FRANKS

G. KASSIOTIS


## Combine all the data cleaning process together

In [8]:
def data_prepare(df):
    df['Address'] = df['Address'].str.upper()#uppercase address 
    df['PI_Address'] = df['Address'].apply(lambda x: x.split(';')[-1]) #only keep PI's address
    df['Country'] = df['PI_Address'].apply(lambda x: x.split(', ')[-1]) #get PI's country
    df['Country'] = df['Country'].apply(lambda x: x.split('.')[:1][0]) # remove email info, only keep country
    import re
    df['Country'] = df['Country'].apply(lambda x: re.sub(r"[^A-Z]","",x))
    
    from re import search
    for i in range(df.shape[0]):
        if df['Country'][i].find('USA') != -1:
            df['Country'][i] = 'USA'

        if df['Country'][i].find('CANADA') != -1:
            df['Country'][i] = 'CANADA'
        if df['Country'][i] in ['TORONTO']:
            df['Country'][i] = 'CANADA'        
        
        if df['Country'][i].find('NETHERLAND') != -1:
            df['Country'][i] = 'NETHERLANDS'
        if df['Country'][i].find('JAPAN') != -1:
            df['Country'][i] = 'JAPAN'
        if df['Country'][i].find('KOREA') != -1:
            df['Country'][i] = 'KOREA'
        if df['Country'][i] == 'SEOULNATIONALUNIVERSITY':
            df['Country'][i] = 'KOREA'    
        
        if df['Country'][i].find('CHINA') != -1:
            df['Country'][i] = 'CHINA'
        if df['Country'][i].find('HONGKONG') != -1:
            df['Country'][i] = 'CHINA'
        
        if df['Country'][i] in ['HONGKONG', 'TAIWAN', 'P', 'PR','PRC', 'BEIJING','PEOPLES']:
            df['Country'][i] = 'CHINA'
        if df['Country'][i].find('RUSSIA') != -1:
            df['Country'][i] = 'RUSSIA'

        if df['Country'][i].find('SINGAPORE') != -1:
            df['Country'][i] = 'SINGAPORE' 
        if df['Country'][i].find('FRANCE') != -1:
            df['Country'][i] = 'FRANCE' 
        if df['Country'][i].find('PARIS') != -1:
            df['Country'][i] = 'FRANCE'
    
        if df['Country'][i].find('GERMANY') != -1:
            df['Country'][i] = 'GERMANY' 
        if df['Country'][i] in ['HEIDELBERG','BERLIN']:
            df['Country'][i] = 'GERMANY'        
        
    
        if df['Country'][i].find('GEORGIA') != -1:
            df['Country'][i] = 'GEORGIA'     
    
    
        if df['Country'][i].find('PANAMA') != -1:
            df['Country'][i] = 'PANAMA'   
        if df['Country'][i].find('SAUDIARABIA') != -1:
            df['Country'][i] = 'SAUDI ARABIA'        
        
        if df['Country'][i] == 'UNITEDARABEMIRATES':
            df['Country'][i] = 'UNITED ARAB EMIRATES'
        if df['Country'][i] == 'BRASIL':
            df['Country'][i] = 'BRAZIL'       
         
        if df['Country'][i].find('UK') != -1:
            df['Country'][i] = 'UK' 
        if df['Country'][i] in ['UNITEDKINGDOM','UNITEDKINGDON', 'CAMBRIDGE','ENGLAND', 'SCOTLAND', 'WALES', 'NORTHERNIRELAND']:
            df['Country'][i] = 'UK'
        if df['Country'][i] in ['KISUMU']:
            df['Country'][i] = 'KENYA'
        if df['Country'][i] in ['UNIVERSIDADNACIONALA']:
            df['Country'][i] = 'COLOMBIA'  
        if df['Country'][i] in ['BLOOMINGTON']:
            df['Country'][i] = 'INDANA' 
             
        if df['Country'][i] in ['AARHUSUNI']:
            df['Country'][i] = 'DENMARK'
        
        if df['Country'][i] == 'NEWZEALAND':
            df['Country'][i] = 'NEW ZEALAND'  
    
        if df['Country'][i] == 'CZECHREPUBLIC':
            df['Country'][i] = 'CZECH REPUBLIC'  
        if df['Country'][i].find('SOUTHAFRICA') != -1:
            df['Country'][i] = 'SOUTH AFRICA'  
        
        if df['Country'][i] in ['HOWARDHUGHESMEDICALINSTITUTE', 'UCLASCHOOLOFMEDICINE','CALIFORNIA', 'ILLINOIS',
                            'UNITEDSTATES', 'TEXAS','MARYLAND','INSTITUTEOFGEOPHYSICSANDPLANETARY','CALIFORNIAI',
                            'UNIVERSITYOFTEXASATAUSTIN','GALVESTON','BIRMINGHAM','SEATTLE','SANTABARBARA',
                            'UNIVERSITYOFCALIFORNIAATSANFRANCISCO','UNIVERSITYOFCALIFORNIAATSANFRANCISCOUCSF',
                            'UNIVERSITYOFCALIF','NEWYORK','BOSTON','IRVINE','HOUSTON','IOWACITY','ITHACANY',
                            'SANFRANCISCO','SANTACRUZ','WORCESTER','OMAHA','DALLAS','SANDIEGO','CALI',
                            'LAWRENCELIVERMORENATIONALL','RI','MADISON','BOULDER','BERKELE','UNIVERSITYOFWISCONSINMADISON',
                            'UNIVERSITYOFCALIFORNIAATBERKELEY','CHAPELHILL','RIVERSIDE','AUSTIN','LAJOLLA','MINNEAPOLIS',
                            'CHARLOTTESVILLE','CHILDRENSHOSPITALOFPHILADELPHIA','SALTLAKECITY','EASTLANSING','DENVER',
                            'RALEIGH','CALIFORNIAINSTITUTEOFTECH','PISCATAWAY','UCLA','BERKELY','BERKELEY','GAINESVILLE','RENO',
                            'CHARLESTOWN','LEXINGTON','SOUTHSANFRANCISCO','LOSANGELES','UNIVERSITYOFCALIFORNIA',
                            'ATLANTA','PASADENA','ANNARBOR','EUGENE','STONYBROOK',
                            'HI','PA','OH','CO','CA','MA','MD','NY','NH','MN','TN','MI','NH','MN','DC','IL',
                            'IN','WI','NJ','TX','NC','MO','VA','WA','CT','US','DE','UTAH','AL','FL','DAVIS',
                           'NM', 'LA','OR','GA','OK','NE','AZ']:
            df['Country'][i] = 'USA'            
    
    
    df['Year'] = df['Year'].apply(lambda x: int(x))
    df['OtherAuthor'] = df['Author'].apply(lambda x: x.split("and")[0])
    df['PI'] = df['Author'].apply(lambda x: x.split("and")[-1].strip())
    return df

    Reset the dataset and clean it using data_prepare() function to check whether the function works well

In [9]:
colnames = ['Author', 'Year', 'Date', 'Title', 'Journal', 'Abstract','URL','Notes', 'Address']
df = pd.read_csv('SCIENCE FULL.csv', header=None, names = colnames).dropna().reset_index(drop = True)
print(df.shape)
df.head()

(26780, 9)


,Author,Year,Date,Title,Journal,Abstract,URL,Notes,Address
0,"Z. GALCHEVA-GARGOVA, K. N. KONSTANTINOV, I. H....",1996,Jun 21,Binding of zinc finger protein ZPR1 to the epi...,Science,ZPR1 is a zinc finger protein that binds to th...,https://www.ncbi.nlm.nih.gov/pubmed/8650580,Galcheva-Gargova Z;Konstantinov K N;Wu I H...,Department of Biochemistry and Molecular Biolo...
1,"A. GENIN, J. S. JAFFE, R. REEF, C. RICHTER and...",2005,May 6,Swimming against the flow: a mechanism of zoop...,Science,Zooplankton reside in a constantly flowing env...,https://www.ncbi.nlm.nih.gov/pubmed/15879218,Genin Amatzia;Jaffe Jules S;Reef Ruth;Rich...,Interuniversity Institute for Marine Sciences ...
2,"K. W. NG, N. FAULKNER, G. H. CORNISH, A. ROSA,...",2020,Dec 11,Preexisting and de novo humoral immunity to SA...,Science,Zoonotic introduction of novel coronaviruses m...,https://www.ncbi.nlm.nih.gov/pubmed/33159009,Ng Kevin W;Faulkner Nikhil;Cornish Georgin...,"Retroviral Immunology, The Francis Crick Insti..."
3,"K. N. ALLERS, J. M. VOS, B. A. BILLER and P. K...",2020,Apr 10,A measurement of the wind speed on a brown dwarf,Science,Zonal (latitudinal) winds dominate the bulk fl...,https://www.ncbi.nlm.nih.gov/pubmed/32273464,Allers Katelyn N;Vos Johanna M;Biller Beth...,"Department of Physics and Astronomy, Bucknell ..."
4,"L. M. SANCHEZ, A. J. CHIRINO and PJ BJORKMAN",1999,Mar 19,"Crystal structure of human ZAG, a fat-depletin...",Science,Zn-alpha2-glycoprotein (ZAG) is a soluble prot...,https://www.ncbi.nlm.nih.gov/pubmed/10206894,Sanchez L M;Chirino A J;Bjorkman P j;eng;S...,"Division of Biology, California Institute of T..."


In [10]:
df_c = data_prepare(df)
print(df_c.shape)
df_c.head()

(26780, 13)


,Author,Year,Date,Title,Journal,Abstract,URL,Notes,Address,PI_Address,Country,OtherAuthor,PI
0,"Z. GALCHEVA-GARGOVA, K. N. KONSTANTINOV, I. H....",1996,Jun 21,Binding of zinc finger protein ZPR1 to the epi...,Science,ZPR1 is a zinc finger protein that binds to th...,https://www.ncbi.nlm.nih.gov/pubmed/8650580,Galcheva-Gargova Z;Konstantinov K N;Wu I H...,DEPARTMENT OF BIOCHEMISTRY AND MOLECULAR BIOLO...,DEPARTMENT OF BIOCHEMISTRY AND MOLECULAR BIOLO...,USA,"Z. GALCHEVA-GARGOVA, K. N. KONSTANTINOV, I. H....",R. J. DAVIS
1,"A. GENIN, J. S. JAFFE, R. REEF, C. RICHTER and...",2005,May 6,Swimming against the flow: a mechanism of zoop...,Science,Zooplankton reside in a constantly flowing env...,https://www.ncbi.nlm.nih.gov/pubmed/15879218,Genin Amatzia;Jaffe Jules S;Reef Ruth;Rich...,INTERUNIVERSITY INSTITUTE FOR MARINE SCIENCES ...,INTERUNIVERSITY INSTITUTE FOR MARINE SCIENCES ...,ISRAEL,"A. GENIN, J. S. JAFFE, R. REEF, C. RICHTER",P. J. FRANKS
2,"K. W. NG, N. FAULKNER, G. H. CORNISH, A. ROSA,...",2020,Dec 11,Preexisting and de novo humoral immunity to SA...,Science,Zoonotic introduction of novel coronaviruses m...,https://www.ncbi.nlm.nih.gov/pubmed/33159009,Ng Kevin W;Faulkner Nikhil;Cornish Georgin...,"RETROVIRAL IMMUNOLOGY, THE FRANCIS CRICK INSTI...","DEPARTMENT OF MEDICINE, FACULTY OF MEDICINE, I...",UK,"K. W. NG, N. FAULKNER, G. H. CORNISH, A. ROSA,...",G. KASSIOTIS
3,"K. N. ALLERS, J. M. VOS, B. A. BILLER and P. K...",2020,Apr 10,A measurement of the wind speed on a brown dwarf,Science,Zonal (latitudinal) winds dominate the bulk fl...,https://www.ncbi.nlm.nih.gov/pubmed/32273464,Allers Katelyn N;Vos Johanna M;Biller Beth...,"DEPARTMENT OF PHYSICS AND ASTRONOMY, BUCKNELL ...","AMERICAN ASTRONOMICAL SOCIETY, WASHINGTON, DC ...",USA,"K. N. ALLERS, J. M. VOS, B. A. BILLER",P. K. G. WILLIAMS
4,"L. M. SANCHEZ, A. J. CHIRINO and PJ BJORKMAN",1999,Mar 19,"Crystal structure of human ZAG, a fat-depletin...",Science,Zn-alpha2-glycoprotein (ZAG) is a soluble prot...,https://www.ncbi.nlm.nih.gov/pubmed/10206894,Sanchez L M;Chirino A J;Bjorkman P j;eng;S...,"DIVISION OF BIOLOGY, CALIFORNIA INSTITUTE OF T...","DIVISION OF BIOLOGY, CALIFORNIA INSTITUTE OF T...",USA,"L. M. SANCHEZ, A. J. CHIRINO",PJ BJORKMAN


    Everything looks good.

## Time to label PI as New, Chaperoned and Established

    Labels will be assigned using the following logic:

    A PI never published a paper before in Science neither as last nor non-last authors will be New.
    A PI only published papers as a non-last author before in Science will be Chaperoned.
    A PI published paper as the last author before in Science will be Established.

    Labels are assigned year by year. For example, if I want to label PIs in the year 2000: the previous years' PIs list and non-PI authors list will be collected, then PIs will be labeled based on the logic mentioned above. 
    In case some New or Chaperoned PIs published more than one paper in one year. Only the first publication will be labeled as New or Chaperoned, and other publications later than that will be labeled as Established.

    Moreover, I am also interested in how many papers will a non-last author have to publish until he/she becomes chaperoned. And this value will be collected simply by counting the frequency of that author in the previous Non-last author list, in other words: how many times he/she appeared as non-last authors. 

In [12]:
def pi_label(data, year):
    data['Year'] = data['Year'].apply(lambda x: int(x))
    # prepare 
    df1 = data.loc[data['Year'] < year].reset_index(drop = True)
    df1 = data_prepare(df1)
    
    Author_List = []
    for i in range(df1.shape[0]):
        L = df1.OtherAuthor[i].split(',')
        for elem in L:
            Author_List.extend(elem.strip().split(';'))
    PI_list = list(df1.PI)

    df2 = data.loc[data['Year'] == year].reset_index(drop = True)
    df2 = data_prepare(df2)
       
    df2['Label'] = pd.Series(index=df2.index)
    df2['NonPI_Pubs'] = pd.Series(index=df2.index)
    

    for i in range(df2.shape[0]):
        if df2['PI'][i] in PI_list:
            df2['Label'][i] = 'Established'
        if (df2['PI'][i] not in PI_list) and (df2['PI'][i] not in Author_List):
            df2['Label'][i] = 'New'    
        if (df2['PI'][i] in Author_List) and (df2['PI'][i] not in PI_list):
            df2['Label'][i] = 'Chaperoned'
            df2['NonPI_Pubs'][i] = Author_List.count(df2['PI'][i])
    #in case a new or chaperoned PI published more than one paper at that year, 
    #only the first publication will be labeled as New or Chaperoned others will be Established
        if (df2.loc[df2.PI == df2.PI[i]].shape[0] > 1):
            List = df2.loc[df2.PI == df2.PI[i]].index
            Establish_list = List[1:]
            df2.Label[Establish_list] = 'Established'          
            
    #import numpy as np
    #df2['ToPI_Perc'] = round(df2[df2.Label == 'Chaperoned'].shape[0] / len(np.unique(Author_List)) , 4)
            
    df2 = df2[['Journal', 'Year', 'PI', 'Label', 'NonPI_Pubs' ,'PI_Address', 'Country', 'Title']]
                        
    #print(' ')
    print('Label year:', year, 'completed')
    #print(df2.Label.value_counts())
    return df2

### Create a loop to label PIs from year 2000 to 2020 

In [13]:
df_labeled = pd.DataFrame()
for year in range(2000, 2021):
    df_year = pi_label(df_c, year)
    df_labeled = df_labeled.append(df_year, True) 

Label year: 2000 completed
Label year: 2001 completed
Label year: 2002 completed
Label year: 2003 completed
Label year: 2004 completed
Label year: 2005 completed
Label year: 2006 completed
Label year: 2007 completed
Label year: 2008 completed
Label year: 2009 completed
Label year: 2010 completed
Label year: 2011 completed
Label year: 2012 completed
Label year: 2013 completed
Label year: 2014 completed
Label year: 2015 completed
Label year: 2016 completed
Label year: 2017 completed
Label year: 2018 completed
Label year: 2019 completed
Label year: 2020 completed


In [14]:
df_labeled.Label.value_counts()

Established    7651
New            7112
Chaperoned     4163
Name: Label, dtype: int64

    Label function works good as well. 

### Read all data and perform cleansing and labeling 

In [16]:
colnames = ['Author', 'Year', 'Date', 'Title', 'Journal', 'Abstract','URL','Notes', 'Address']
dfs = pd.read_csv('SCIENCE FULL.csv', header=None, names = colnames).dropna().reset_index(drop = True)
dfn = pd.read_csv('NATURE FULL.csv', header=None, names = colnames).dropna().reset_index(drop = True)
dfc = pd.read_csv('CELL FULL.csv', header=None, names = colnames).dropna().reset_index(drop = True)

In [17]:
print('Numbers of publications of Science till end of 2020: ', dfs.shape)
print('Numbers of publications of Nature till end of 2020: ', dfn.shape)
print('Numbers of publications of Cell till end of 2020: ', dfc.shape)

Numbers of publications of Science till end of 2020:  (26780, 9)
Numbers of publications of Nature till end of 2020:  (26630, 9)
Numbers of publications of Cell till end of 2020:  (14218, 9)


#### Cleaning (mining PI, non-PI authors and countries of PIs)

In [18]:
dfs_clean = data_prepare(dfs)
dfn_clean = data_prepare(dfn)
dfc_clean = data_prepare(dfc)

dfc_clean.head()

,Author,Year,Date,Title,Journal,Abstract,URL,Notes,Address,PI_Address,Country,OtherAuthor,PI
0,E. L. FERGUSON and K. V. ANDERSON,1992,Oct 30,Decapentaplegic acts as a morphogen to organiz...,Cell,Zygotic expression of the Drosophila TGF beta ...,https://www.ncbi.nlm.nih.gov/pubmed/1423606,Ferguson E L;Anderson K V;eng;GM 35437/GM/N...,"DEPARTMENT OF MOLECULAR AND CELL BIOLOGY, UNIV...","DEPARTMENT OF MOLECULAR AND CELL BIOLOGY, UNIV...",USA,E. L. FERGUSON,K. V. ANDERSON
1,"L. HAN, M. MONNE, H. OKUMURA, T. SCHWEND, A. L...",2010,Oct 29,Insights into egg coat assembly and egg-sperm ...,Cell,"ZP3, a major component of the zona pellucida (...",https://www.ncbi.nlm.nih.gov/pubmed/20970175,Han Ling;Monne Magnus;Okumura Hiroki;Schwe...,"DEPARTMENT OF BIOSCIENCES AND NUTRITION, CENTE...","DEPARTMENT OF BIOSCIENCES AND NUTRITION, CENTE...",SWEDEN,"L. HAN, M. MONNE, H. OKUMURA, T. SCHWEND, A. L...",L. JOVINE
2,"J. N. MANDL, R. AHMED, L. B. BARREIRO, P. DASZ...",2015,Jan 15,Reservoir host immune responses to emerging zo...,Cell,"Zoonotic viruses, such as HIV, Ebola virus, co...",https://www.ncbi.nlm.nih.gov/pubmed/25533784,Mandl Judith N;Ahmed Rafi;Barreiro Luis B;...,"LYMPHOCYTE BIOLOGY SECTION, LABORATORY OF SYST...","MERCK VACCINES, MERCK & CO. INC., WEST POINT, ...",USA,"J. N. MANDL, R. AHMED, L. B. BARREIRO, P. DASZ...",M. B. FEINBERG
3,"M. SYM, J. A. ENGEBRECHT and G. S. ROEDER",1993,Feb 12,ZIP1 is a synaptonemal complex protein require...,Cell,ZIP1 is a novel meiosis-specific gene required...,https://www.ncbi.nlm.nih.gov/pubmed/7916652,Sym M;Engebrecht J A;Roeder G S;eng;2 T32 ...,"DEPARTMENT OF BIOLOGY, YALE UNIVERSITY, NEW HA...","DEPARTMENT OF BIOLOGY, YALE UNIVERSITY, NEW HA...",CONNECTICUT,"M. SYM, J. A. ENGEBRECHT",G. S. ROEDER
4,"J. WANG, M. BARDELLI, D. A. ESPINOSA, M. PEDOT...",2017,Sep 21,A Human Bi-specific Antibody against Zika Viru...,Cell,"Zika virus (ZIKV), a mosquito-borne flavivirus...",https://www.ncbi.nlm.nih.gov/pubmed/28938115,Wang Jiaqi;Bardelli Marco;Espinosa Diego A...,"PROGRAM IN EMERGING INFECTIOUS DISEASES, DUKE-...",HUMABS BIOMED SA A SUBSIDIARY OF VIR BIOTECHNO...,SWITZERLAND,"J. WANG, M. BARDELLI, D. A. ESPINOSA, M. PEDOT...",D. CORTI


#### Label Science PIs from year 2000 to 2020

In [19]:
dfs_labeled = pd.DataFrame()
for year in range(2000, 2021):
    df_year = pi_label(dfs_clean, year)
    dfs_labeled = dfs_labeled.append(df_year, True) 

Label year: 2000 completed
Label year: 2001 completed
Label year: 2002 completed
Label year: 2003 completed
Label year: 2004 completed
Label year: 2005 completed
Label year: 2006 completed
Label year: 2007 completed
Label year: 2008 completed
Label year: 2009 completed
Label year: 2010 completed
Label year: 2011 completed
Label year: 2012 completed
Label year: 2013 completed
Label year: 2014 completed
Label year: 2015 completed
Label year: 2016 completed
Label year: 2017 completed
Label year: 2018 completed
Label year: 2019 completed
Label year: 2020 completed


#### Label Nature PIs from year 2000 to 2020

In [20]:
dfn_labeled = pd.DataFrame()
for year in range(2000, 2021):
    df_year = pi_label(dfn_clean, year)
    dfn_labeled = dfn_labeled.append(df_year, True)

Label year: 2000 completed
Label year: 2001 completed
Label year: 2002 completed
Label year: 2003 completed
Label year: 2004 completed
Label year: 2005 completed
Label year: 2006 completed
Label year: 2007 completed
Label year: 2008 completed
Label year: 2009 completed
Label year: 2010 completed
Label year: 2011 completed
Label year: 2012 completed
Label year: 2013 completed
Label year: 2014 completed
Label year: 2015 completed
Label year: 2016 completed
Label year: 2017 completed
Label year: 2018 completed
Label year: 2019 completed
Label year: 2020 completed


#### Label Cell PIs from year 2000 to 2020

In [21]:
dfc_labeled = pd.DataFrame()
for year in range(2000, 2021):
    df_year = pi_label(dfc_clean, year)
    dfc_labeled = dfc_labeled.append(df_year, True)
dfc_labeled.head()

Label year: 2000 completed
Label year: 2001 completed
Label year: 2002 completed
Label year: 2003 completed
Label year: 2004 completed
Label year: 2005 completed
Label year: 2006 completed
Label year: 2007 completed
Label year: 2008 completed
Label year: 2009 completed
Label year: 2010 completed
Label year: 2011 completed
Label year: 2012 completed
Label year: 2013 completed
Label year: 2014 completed
Label year: 2015 completed
Label year: 2016 completed
Label year: 2017 completed
Label year: 2018 completed
Label year: 2019 completed
Label year: 2020 completed


,Journal,Year,PI,Label,NonPI_Pubs,PI_Address,Country,Title
0,Cell,2000,S. K. BURLEY,Established,NaN,LABORATORIES OF MOLECULAR BIOPHYSICS THE ROCKE...,USA,X-Ray structures of the universal translation ...
1,Cell,2000,S. H. SNYDER,Established,NaN,"JOHNS HOPKINS UNIVERSITY SCHOOL OF MEDICINE, D...",USA,Pike. A nuclear gtpase that enhances PI3kinase...
2,Cell,2000,R. PATIENT,New,NaN,"INSTITUTE OF GENETICS, UNIVERSITY OF NOTTINGHA...",UK,Distinct origins of adult and embryonic blood ...
3,Cell,2000,M. ROSBASH,Established,NaN,"HOWARD HUGHES MEDICAL INSTITUTE, NATIONAL SCIE...",USA,The Drosophila takeout gene is a novel molecul...
4,Cell,2000,B. E. UHLIN,Established,NaN,"DEPARTMENT OF MICROBIOLOGY, UMEA UNIVERSITY, S...",SWEDEN,Nucleoid proteins stimulate stringently contro...


In [35]:
print('Numbers of publications of Science from 2000 to 2020: ', dfs_labeled.shape)
print('Numbers of publications of Nature from 2000 to 2020: ', dfn_labeled.shape)
print('Numbers of publications of Cell from 2000 to 2020: ', dfc_labeled.shape)

Numbers of publications of Science from 2000 to 2020:  (18926, 9)
Numbers of publications of Nature from 2000 to 2020:  (19491, 9)
Numbers of publications of Cell from 2000 to 2020:  (9749, 9)


### Count number of publications of each PI in 20 years

In [23]:
def count_pub(df):
    df['Total_Pubs'] = pd.Series(index=df.index)
    PI_list = list(df.PI)
    for i in range(df.shape[0]):
        df['Total_Pubs'][i] = PI_list.count(df['PI'][i])
    return df
dfs_labeled = count_pub(dfs_labeled)
dfn_labeled = count_pub(dfn_labeled)
dfc_labeled = count_pub(dfc_labeled)

dfc_labeled.head()

,Journal,Year,PI,Label,NonPI_Pubs,PI_Address,Country,Title,Total_Pubs
0,Cell,2000,S. K. BURLEY,Established,NaN,LABORATORIES OF MOLECULAR BIOPHYSICS THE ROCKE...,USA,X-Ray structures of the universal translation ...,7.0
1,Cell,2000,S. H. SNYDER,Established,NaN,"JOHNS HOPKINS UNIVERSITY SCHOOL OF MEDICINE, D...",USA,Pike. A nuclear gtpase that enhances PI3kinase...,5.0
2,Cell,2000,R. PATIENT,New,NaN,"INSTITUTE OF GENETICS, UNIVERSITY OF NOTTINGHA...",UK,Distinct origins of adult and embryonic blood ...,1.0
3,Cell,2000,M. ROSBASH,Established,NaN,"HOWARD HUGHES MEDICAL INSTITUTE, NATIONAL SCIE...",USA,The Drosophila takeout gene is a novel molecul...,4.0
4,Cell,2000,B. E. UHLIN,Established,NaN,"DEPARTMENT OF MICROBIOLOGY, UMEA UNIVERSITY, S...",SWEDEN,Nucleoid proteins stimulate stringently contro...,2.0


#### Combine all labeled data together

In [24]:
df_with_label = pd.concat([dfs_labeled, dfn_labeled, dfc_labeled]).reset_index(drop = True)
df_with_label.head()

,Journal,Year,PI,Label,NonPI_Pubs,PI_Address,Country,Title,Total_Pubs
0,Science,2000,B. J. SAHAKIAN,New,NaN,"DEPARTMENT OF EXPERIMENTAL PSYCHOLOGY, UNIVERS...",UK,Neuroscience. Boosting working memory,3.0
1,Science,2000,J. HARDIN,Chaperoned,1.0,DEPARTMENT OF ZOOLOGY AND PROGRAM IN CELLULAR ...,USA,Development. A degrading way to make an organ,1.0
2,Science,2000,O. A. MACDOUGALD,New,NaN,"DEPARTMENT OF PHYSIOLOGY, DEPARTMENT OF PATHOL...",USA,Inhibition of adipogenesis by Wnt signaling,1.0
3,Science,2000,M. MLODZIK,Chaperoned,1.0,"EUROPEAN MOLECULAR BIOLOGY LABORATORY, DEVELOP...",GERMANY,Signaling specificity by Frizzled receptors in...,1.0
4,Science,2000,M. BRONNER-FRASER,Established,NaN,"DIVISION OF BIOLOGY 139-74, CALIFORNIA INSTITU...",USA,"N-Cadherin, a cell adhesion molecule involved ...",2.0


#### One more step to check PIs' countries

    Remember, when I do country mining initially, I didn't get a super 'clean' dataset, and it is time to get it done.
    Firstly, let's check the top 50 countries according to their numbers of publications.

In [25]:
df_with_label.Country.value_counts()[:51]

USA                     27729
UK                       4225
GERMANY                  3108
JAPAN                    1707
FRANCE                   1608
CHINA                    1272
SWITZERLAND              1253
CANADA                   1186
NETHERLANDS               856
AUSTRALIA                 706
ITALY                     464
ISRAEL                    463
SWEDEN                    416
SPAIN                     414
AUSTRIA                   364
DENMARK                   319
KOREA                     238
BELGIUM                   236
SINGAPORE                 151
NORWAY                    117
FINLAND                   100
NEW ZEALAND                86
INDIA                      77
RUSSIA                     73
IRELAND                    69
SOUTH AFRICA               68
BRAZIL                     58
HUNGARY                    54
PORTUGAL                   51
ARGENTINA                  43
GREECE                     40
CZECH REPUBLIC             32
CHILE                      28
ICELAND   

#### Note
    Countries with labels 'U' and '' have a relatively large number of publications. However, they are definitely not correct labels of Countries. I will print their addresses out and correct them manually. 

In [26]:
for i in df_with_label.loc[df_with_label.Country == 'U'].index:
    print(i, df_with_label.loc[df_with_label.Country == 'U'].PI_Address[i])
    print('')   

366 LONDON SCHOOL OF HYGIENE & TROPICAL MEDICINE, LONDON WC1E 7HT, U.K. CHRIS.CURTIS@LSHTM.AC.UK

576 DEPARTMENT OF GEOLOGY AND GEOPHYSICS, DEPARTMENT OF PHYSICS, UNIVERSITY OF CALIFORNIA, BERKELEY, CA 94720, USA. LAWRENCE BERKELEY NATIONAL LABORATORY, BERKELEY, CA 94720, USA. BERKELEY GEOCHRONOLOGY CENTER, 2455 RIDGE ROAD, BERKELEY, CA 94709, U.

3075 DEPARTMENT OF PHYSIOLOGY, UNIVERSITY COLLEGE LONDON, GOWER STREET, LONDON WC1E 6BT, U.K. A.SILVER@UCL.AC.UK

6023 MEDICAL RESEARCH COUNCIL DUNN HUMAN NUTRITION UNIT, WELLCOME TRUST/MRC BUILDING, HILLS ROAD, CAMBRIDGE CB2 2XY, U.K. SAZANOV@MRC-DUNN.CAM.AC.UK

7066 DEPARTMENT OF IMMUNOLOGY, UNIVERSITY OF CONNECTICUT, FARMINGTON, CT 06030, U.S.A.

9247 INSITUTE OF COGNITIVE NEUROSCIENCE, UNIVERSITY COLLEGE LONDON, 17 QUEEN SQUARE, LONDON WC1N 3AR, U.K.

13444 INTRAMURAL RESEARCH PROGRAM OF THE EUNICE KENNEDY SHRIVER NATIONAL INSTITUTE OF CHILD HEALTH AND HUMAN DEVELOPMENT, NATIONAL INSTITUTES OF HEALTH, BETHESDA, MD, USA. COMMISSIONED CORPS

In [27]:
df_with_label.Country[366, 3075, 6023, 7247, 15892, 20574] = 'UK'
df_with_label.Country[576, 7066, 13444, 15191, 45332] = 'USA'

In [28]:
for i in df_with_label.loc[df_with_label.Country == ''].index:
    print(i, df_with_label.loc[df_with_label.Country == ''].PI_Address[i])
    print('')

2751 GEOGRAPHY AND CRUSTAL DYNAMICS RESEARCH CENTER, GEOGRAPHICAL SURVEY INSTITUTE OF JAPAN, KITASATO-1, TSUKUBA, IBARAKI, JAPAN, 305-0013. OZAWA@GSI.GO.JP

4410 CENTER FOR THE STUDY OF SYNTHETIC ANTIGENS, FACULTAD DE QUIMICA, UNIVERSIDAD DE LA HABANA, CIUDAD HABANA, CUBA, 10400. VICENTE@FQ.UH.CU

12347 DEPARTMENTS OF NEUROLOGY AND NEUROBIOLOGY, UNIVERSITY OF CALIFORNIA AT LOS ANGELES, 90095.

32218 DEPARTMENT OF MICROBIOLOGY AND IMMUNOLOGY, BROWN UNIVERSITY, 171 MEETING ST, PROVIDENCE, RI, 02912.

33301 DEPARTMENT OF MOLECULAR AND CELLULAR BIOLOGY AND CENTER FOR BRAIN SCIENCE, HARVARD UNIVERSITY, CAMBRIDGE, MA, 02138.

33908 SYSTEMS NEUROBIOLOGY LABORATORY, DEPARTMENT OF NEUROBIOLOGY, DAVID GEFFEN SCHOOL OF MEDICINE, UNIVERSITY OF CALIFORNIA LOS ANGELES, LOS ANGELES, CA, 90095.

36566 

41339 THE JOHN CURTIN SCHOOL OF MEDICAL RESEARCH, THE AUSTRALIAN NATIONAL UNIVERSITY, P.O. BOX 334, CANBERRA, THE AUSTRALIAN CAPITAL TERRITORY, AUSTRALIA, 2601. DAVID.TREMETHICK@ANU.EDU.AU

44361 DEPAR

In [29]:
df_with_label.Country[2751] = 'JAPAN'
df_with_label.Country[4410] = 'CUBA'
df_with_label.Country[12347, 32218, 33301, 33908, 44361, 45351] = 'USA'
df_with_label.Country[41339] = 'AUSTRALIA'

    Check again 

In [30]:
df_with_label.Country.value_counts()[:51]

USA                     27739
UK                       4231
GERMANY                  3108
JAPAN                    1708
FRANCE                   1608
CHINA                    1272
SWITZERLAND              1253
CANADA                   1186
NETHERLANDS               856
AUSTRALIA                 707
ITALY                     464
ISRAEL                    463
SWEDEN                    416
SPAIN                     414
AUSTRIA                   364
DENMARK                   319
KOREA                     238
BELGIUM                   236
SINGAPORE                 151
NORWAY                    117
FINLAND                   100
NEW ZEALAND                86
INDIA                      77
RUSSIA                     73
IRELAND                    69
SOUTH AFRICA               68
BRAZIL                     58
HUNGARY                    54
PORTUGAL                   51
ARGENTINA                  43
GREECE                     40
CZECH REPUBLIC             32
CHILE                      28
ICELAND   

    Check countries 51 to 100

In [31]:
df_with_label.Country.value_counts()[51:101]

INDONESIA                                   5
AMSTERDAM                                   5
MADAGASCAR                                  5
MALAYSIA                                    4
MGX                                         4
PERU                                        4
IRAN                                        3
GABON                                       3
EGYPT                                       3
CONNECTICUT                                 3
BELGIQUE                                    3
LNZ                                         3
BULGARIA                                    3
TUNISIA                                     3
LUXEMBOURG                                  3
SERBIA                                      3
HAB                                         2
BANGLADESH                                  2
ROC                                         2
LITHUANIA                                   2
BHJ                                         2
FAROEISLANDS                      

    There are still many incorrect countries, and most of them have a frequency of less than 10. Cleaning all of them will be time-consuming, and a better plan is to gather them together.

In [32]:
vc = df_with_label.Country.value_counts()
for i in range(df_with_label.shape[0]):
    if df_with_label.Country[i] in vc[vc < 10].index:
        df_with_label.Country[i] = 'OTHERS'
df_with_label.Country.value_counts()

USA                     27739
UK                       4231
GERMANY                  3108
JAPAN                    1708
FRANCE                   1608
CHINA                    1272
SWITZERLAND              1253
CANADA                   1186
NETHERLANDS               856
AUSTRALIA                 707
ITALY                     464
ISRAEL                    463
SWEDEN                    416
SPAIN                     414
OTHERS                    391
AUSTRIA                   364
DENMARK                   319
KOREA                     238
BELGIUM                   236
SINGAPORE                 151
NORWAY                    117
FINLAND                   100
NEW ZEALAND                86
INDIA                      77
RUSSIA                     73
IRELAND                    69
SOUTH AFRICA               68
BRAZIL                     58
HUNGARY                    54
PORTUGAL                   51
ARGENTINA                  43
GREECE                     40
CZECH REPUBLIC             32
CHILE     

## Data ready for Tableau 🐣

In [33]:
df_with_label.to_csv('all pubs with label.csv', index = False)

In [34]:
import xlsxwriter
df_with_label.to_excel('all pubs with label.xlsx' , engine='xlsxwriter')